In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import ablang2

# **0. Sequence input and its format**

AbLang2 takes as input either the individual heavy variable domain (VH), light variable domain (VL), or the full variable domain (Fv).

Each record (antibody) needs to be a list with the VH as the first element and the VL as the second. If either the VH or VL is not known, leave an empty string.

An asterisk (\*) is used for masking. It is recommended to mask residues which you are interested in mutating.

**NB:** It is important that the VH and VL sequence is ordered correctly.

In [2]:
seq1 = [
    'EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS', # VH sequence
    'DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK' # VL sequence
]
seq2 = [
    'EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTT',
    'PVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
]
seq3 = [
    'EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    '' # The VL sequence is not known, so an empty string is left instead. 
]
seq4 = [
    '',
    'DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
]
seq5 = [
    'EVQ***SGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCAR**PGHGAAFMDVWGTGTTVTVSS', # (*) is used to mask certain residues
    'DIQLTQSPLSLPVTLGQPASISCRSS*SLEASDTNIYLSWFQQRPGQSPRRLIYKI*NRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
]

all_seqs = [seq1, seq2, seq3, seq4, seq5]
only_both_chains_seqs = [seq1, seq2, seq5]

# **1. How to use AbLang2**

AbLang2 can be downloaded and used in its raw form as seen below. For convenience, we have also developed different "modes" which can be used for specific use cases (see Section 2) 

In [3]:
# Download and initialise the model
ablang = ablang2.pretrained(model_to_use='ablang2-paired', random_init=False, ncpu=1, device='cpu')

# Tokenize input sequences
seq = f"{seq1[0]}|{seq1[1]}" # VH first, VL second, with | used to separated the two sequences 
tokenized_seq = ablang.tokenizer([seq], pad=True, w_extra_tkns=False, device="cpu")
        
# Generate rescodings
with torch.no_grad():
    rescoding = ablang.AbRep(tokenized_seq).last_hidden_states

# Generate logits/likelihoods
with torch.no_grad():
    likelihoods = ablang.AbLang(tokenized_seq)

# **2. Different modes for specific usecases**

AbLang2 has already been implemented for a variety of different usecases. The benefit of these modes is that they handle extra tokens such as start, stop and separation tokens.

1. seqcoding: Generates sequence representations for each sequence
2. rescoding: Generates residue representations for each residue in each sequence
3. likelihood: Generates likelihoods for each amino acid at each position in each sequence
4. probability: Generates probabilities for each amino acid at each position in each sequence
5. pseudo_log_likelihood: Returns the pseudo log likelihood for a sequence (based on masking each residue one at a time)
6. confidence: Returns a fast calculation of the log likelihood for a sequence (based on a single pass with no masking)
7. restore: Restores masked residues

### **AbLang2 can also align the resulting representations using ANARCI**

This can be done for 'rescoding', 'likelihood', and 'probability'. This is done by setting the argument "align=True".

**NB**: Align can only be used on input with the same format, i.e. either all heavy, all light, or all both heavy and light.

### **The align argument can also be used to restore variable missing lengths**

For this, use "align=True" with the 'restore' mode.

In [4]:
ablang = ablang2.pretrained()

valid_modes = [
    'seqcoding', 'rescoding', 'likelihood', 'probability',
    'pseudo_log_likelihood', 'confidence', 'restore' 
]

## **seqcoding** 

The seqcodings represents each sequence as a 480 sized embedding. It is derived from averaging across each rescoding embedding for a given sequence, including extra tokens. 

**NB:** Seqcodings can also be derived in other ways like using the sum or averaging across only parts of the input such as the CDRs. For such cases please use and adapt the below rescoding.

In [5]:
ablang(all_seqs, mode='seqcoding')

array([[-0.25206307,  0.18189631,  0.00887138, ...,  0.15365519,
        -0.14508606, -0.1338132 ],
       [-0.25149408,  0.20864546,  0.07518199, ...,  0.19478273,
        -0.15227766, -0.08241639],
       [-0.27468954,  0.16507216,  0.08667152, ...,  0.18776285,
        -0.14165084, -0.16389883],
       [-0.19822123,  0.16841072, -0.0492593 , ...,  0.11400165,
        -0.1472368 , -0.09713164],
       [-0.29553185,  0.17239205,  0.05676922, ...,  0.15943633,
        -0.16615381, -0.15569785]])

## **rescoding / likelihood / probability**

The rescodings represents each residue as a 480 sized embedding. The likelihoods represents each residue as the predicted logits for each character in the vocabulary. The probabilities represents the normalised likelihoods.

**NB:** The output includes extra tokens (start, stop and separation tokens) in the format "<VH_seq>|<VL_seq>". The length of the output is therefore 5 longer than the VH and VL.

**NB:** By default the representations are derived using a single forward pass. To prevent the predicted likelihood and probability to be affected by the input residue at each position, setting the "stepwise_masking" argument to True can be used. This will run a forward pass for each position with the residue at that position masked. This is much slower than running a single forward pass.

In [6]:
ablang(all_seqs, mode='rescoding', stepwise_masking = False)

[array([[-0.4074121 , -0.5118989 ,  0.06096726, ...,  0.32681447,
          0.03920222, -0.3671585 ],
        [-0.5768881 ,  0.3824539 , -0.2179203 , ...,  0.01250257,
         -0.08844507, -0.32367533],
        [-0.1475931 ,  0.39639005, -0.38226977, ..., -0.10119948,
         -0.41469535, -0.00319355],
        ...,
        [-0.14358361,  0.3124388 , -0.30157983, ..., -0.13289262,
         -0.45353422, -0.07878865],
        [ 0.17538936,  0.24394287,  0.20141171, ...,  0.14587358,
         -0.38479036,  0.0740919 ],
        [-0.23031718, -0.35487312,  0.19606815, ..., -0.12833653,
          0.31107306, -0.32651076]], dtype=float32),
 array([[-0.41981825, -0.3666373 ,  0.10595194, ...,  0.39035738,
          0.03823777, -0.36337993],
        [-0.5054134 ,  0.38347128, -0.10992065, ..., -0.05231512,
         -0.1363659 , -0.348301  ],
        [-0.06784608,  0.6934987 , -0.42123976, ..., -0.24805394,
         -0.39583805, -0.10972723],
        ...,
        [-0.2090097 ,  0.29489487, -0.1

## **Align rescoding/likelihood/probability output**

For the 'rescoding', 'likelihood', and 'probability' modes, the output can also be aligned using the argument "align=True".

This is done using the antibody numbering tool ANARCI, and requires manually installing **Pandas** and **[ANARCI](https://github.com/oxpig/ANARCI)**.

**NB**: Align can only be used on input with the same format, i.e. either all heavy, all light, or all both heavy and light.

In [7]:
results = ablang(only_both_chains_seqs, mode='likelihood', align=True)

print(results.number_alignment)
print(results.aligned_seqs)
print(results.aligned_embeds)

['<' '1 ' '2 ' '3 ' '4 ' '5 ' '6 ' '7 ' '8 ' '9 ' '11 ' '12 ' '13 ' '14 '
 '15 ' '16 ' '17 ' '18 ' '19 ' '20 ' '21 ' '22 ' '23 ' '24 ' '25 ' '26 '
 '27 ' '28 ' '29 ' '30 ' '35 ' '36 ' '37 ' '38 ' '39 ' '40 ' '41 ' '42 '
 '43 ' '44 ' '45 ' '46 ' '47 ' '48 ' '49 ' '50 ' '51 ' '52 ' '53 ' '54 '
 '55 ' '56 ' '57 ' '58 ' '59 ' '62 ' '63 ' '64 ' '65 ' '66 ' '67 ' '68 '
 '69 ' '70 ' '71 ' '72 ' '74 ' '75 ' '76 ' '77 ' '78 ' '79 ' '80 ' '81 '
 '82 ' '83 ' '84 ' '85 ' '86 ' '87 ' '88 ' '89 ' '90 ' '91 ' '92 ' '93 '
 '94 ' '95 ' '96 ' '97 ' '98 ' '99 ' '100 ' '101 ' '102 ' '103 ' '104 '
 '105 ' '106 ' '107 ' '108 ' '109 ' '110 ' '111 ' '112A' '112 ' '113 '
 '114 ' '115 ' '116 ' '117 ' '118 ' '119 ' '120 ' '121 ' '122 ' '123 '
 '124 ' '125 ' '126 ' '127 ' '128 ' '>' '|' '<' '1 ' '2 ' '3 ' '4 ' '5 '
 '6 ' '7 ' '8 ' '9 ' '10 ' '11 ' '12 ' '13 ' '14 ' '15 ' '16 ' '17 ' '18 '
 '19 ' '20 ' '21 ' '22 ' '23 ' '24 ' '25 ' '26 ' '27 ' '28 ' '29 ' '30 '
 '31 ' '32 ' '34 ' '35 ' '36 ' '37 ' '38 ' '39 ' '40 

## **Pseudo log likelihood and Confidence scores**

The pseudo log likelihood and confidence represents two methods for calculating the uncertainty for the input sequence.

- pseudo_log_likelihood: For each position, the pseudo log likelihood is calculated when predicting the masked residue. The final score is an average across the whole input. This is similar to the approach taken in the ESM-2 paper for calculating pseudo perplexity [(Lin et al., 2023)](https://doi.org/10.1126/science.ade2574).

- confidence: For each position, the log likelihood is calculated without masking the residue. The final score is an average across the whole input. 

**NB:** The **confidence is fast** to compute, requiring only a single forward pass per input. **Pseudo log likelihood is slow** to calculate, requiring L forward passes per input, where L is the length of the input.

**NB:** It is recommended to use **pseudo log likelihood for final results** and **confidence for exploratory work**.

In [8]:
results = ablang(all_seqs, mode='pseudo_log_likelihood')
np.exp(-results) # convert to pseudo perplexity

array([1.9951935, 2.017602 , 2.1375413, 1.8546418, 2.0021744],
      dtype=float32)

In [9]:
results = ablang(all_seqs, mode='confidence')
np.exp(-results)

array([1.2699332, 1.1272193, 1.3212236, 1.2203737, 1.1848253],
      dtype=float32)

## **restore**

This mode can be used to restore masked residues, and fragmented regions with "align=True". 

In [10]:
restored = ablang(only_both_chains_seqs, mode='restore')
restored

array(['<EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS>|<DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>',
       '<EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTT>|<PVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>',
       '<EVQLVQSGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDPPGHGAAFMDVWGTGTTVTVSS>|<DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>'],
      dtype='<U238')

In [11]:
restored = ablang(only_both_chains_seqs, mode='restore', align = True)
restored

array(['<EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS>|<DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>',
       '<EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS>|<DVVMTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>',
       '<QVQLVQSGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDPPGHGAAFMDVWGTGTTVTVSS>|<DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK>'],
      dtype='<U238')